# Lesson 3 Exercise 1: Three Queries Three Tables
<img src="../images/cassandralogo.png" width="250" height="250">

##### This cell activates cassandra docker, wait 1 minute until it activates

In [ ]:
#Run this cell to start cassandra with docker

!mkdir ny_taxi_cassandra
!docker-compose up -d

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

Wait a minute to run this row

❌ **Possible error:** Unable to connect to any servers', {'*': error(111, "Tried connecting to [('*', 9042)]. Last error: Connection refused

✅ **Solution:** Run this cell again

### Create a connection to the database

In [ ]:
auth_provider = PlainTextAuthProvider(username = 'cassandra', password='admin123')
try:
    cluster1 = Cluster(['127.0.0.1'], port=9042, auth_provider=auth_provider)
    session = cluster1.connect()
    
except Exception as e:
    print(e)    

### Walk through the basics of creating a table in Apache Cassandra, inserting rows of data, and doing a simple CQL query to validate the information. You will practice Denormalization, and the concept of 1 table per query, which is an encouraged practice with Apache Cassandra. 

### Remember, replace ##### with your answer.


Note: __Do not__ click the blue Preview button at the bottom

#### We will use a python wrapper/ python driver called cassandra to run the Apache Cassandra queries. This library should be preinstalled but in the future to install this library you can run this command in a notebook to install locally: 
! pip install cassandra-driver
#### More documentation can be found here:  https://datastax.github.io/python-driver/

### Create a connection to the database

In [31]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'],port=9042) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

In [44]:
try:
    session.execute("""
    drop keyspace udacity"""
)

except Exception as e:
    print(e)

### Create a keyspace to work in

In [45]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [46]:
try:
    result= session.execute("""SELECT * FROM system_schema.keyspaces;
    """
)

except Exception as e:
    print(e)
print(result)

#### Connect to our Keyspace. Compare this to how we had to create a new session in PostgreSQL.  

In [47]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Let's imagine we would like to start creating a Music Library of albums. 

### We want to ask 3 questions of the data
#### 1. Give every album in the music library that was released in a given year
`select * from music_library WHERE YEAR=1970`
#### 2. Give every album in the music library that was created by a given artist  
`select * from artist_library WHERE artist_name="The Beatles"`
#### 3. Give all the information from the music library about a given album
`select * from album_library WHERE album_name="Close To You"`


### Because we want to do three different queries, we will need different tables that partition the data differently. 
<img src="../images/table1.png" width="400" height="100">
<img src="../images/table2.png" width="400" height="100">
<img src="../images/table3.png" width="400" height="100">

### TO-DO: Create the tables. 

### TO-DO: Insert data into the tables

This might have felt unnatural to insert duplicate data into the tables. If I just normalized these tables, I wouldn't have to have extra copies! While this is true, remember there are no `JOINS` in Apache Cassandra. For the benefit of high availibity and scalabity, denormalization must be how this is done. 


### TO-DO: Validate the Data Model

1970 The Beatles Let it Be
1970 The Carpenters Close To You


### Your output should be:
1970 The Beatles Let it Be<br>
1970 The Carpenters Close To You

### TO-DO: Validate the Data Model

The Beatles Rubber Soul 1965
The Beatles Let it Be 1970


### Your output should be:
The Beatles Rubber Soul 1965 <br>
The Beatles Let it Be 1970 

### TO-DO: Validate the Data Model

In [53]:
query = "select * from album_library WHERE album_name='Close To You'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.year, row.album_name)

The Carpenters 1970 Close To You


### Your output should be:
The Carpenters 1970 Close To You

### And finally close the session and cluster connection

In [54]:
session.shutdown()
cluster.shutdown()